# Feedback Prize - Evaluating Student Writing
#### Analyze argumentative writing elements from students grade 6-12
![](https://storage.googleapis.com/kaggle-competitions/kaggle/31779/logos/header.png?t=2021-11-12-22-52-17&quot)

https://drive.google.com/file/d/1r9vxKn5Az3ZBoZ7PgkmIh2ov87axyN8B/view

# Import packages

In [ ]:
# Install packages

In [ ]:
# Import packages
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import re
from tqdm import tqdm
tqdm.pandas()
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly as py
import plotly.graph_objs as go
init_notebook_mode(connected=True)
import torch
import torch.nn as nn
from joblib import Parallel, delayed
from transformers import AutoModelForTokenClassification
from transformers import AutoModel, AutoTokenizer, AutoConfig, TrainingArguments, Trainer, EvalPrediction
import sys
import time
import gc
import pickle
import re
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW
import sklearn
SEED = 42

# Load data

In [ ]:
train_data = pd.read_csv('../input/feedback-prize-2021/train.csv')
submission_data = pd.read_csv('../input/feedback-prize-2021/sample_submission.csv')

# EDA

In [ ]:
print(train_data.shape)
train_data.head()

In [ ]:
print('Discourse types in train:\n', len(train_data['discourse_type'].unique().tolist()), 'types -', train_data['discourse_type'].unique().tolist())

In [ ]:
print('# Splits for first passage:', train_data[train_data['id']==train_data['id'].iloc[0]]['discourse_id'].nunique())
train_data[train_data['id']==train_data['id'].iloc[0]]

In [ ]:
print(submission_data.shape)
submission_data.head()

## Check prediction string

In [ ]:
''.join(train_data[train_data['id']==train_data['id'].unique()[2]]['predictionstring'].tolist())

In [ ]:
len(' '.join(train_data[train_data['id']==train_data['id'].unique()[2]]['discourse_text'].apply(lambda x: re.sub('\n+',' ',x)).str.strip().tolist()).split(' ')), len(' '.join(train_data[train_data['id']==train_data['id'].unique()[2]]['predictionstring'].tolist()).split(' '))

In [ ]:
filepath = '../input/feedback-prize-2021/'
record_id = 'A97DE0D49AEA'
# record_id = '1CEDD5563788'
with open(filepath + 'train/' + record_id+'.txt','r') as f:
    data = f.read()

In [ ]:
train_data[train_data['id']==record_id]

In [ ]:
train_data[train_data['id']==record_id]['discourse_text'].iloc[0]

In [ ]:
re.sub('\n+',' ', data).strip().replace("\xa0"," ").replace('"','')

Combined words create an issue e.g: "DrivingHundreds" in the above passage

Also references are not to be tagged under the 7 classes

In [ ]:
re.sub('\n+',' ',train_data[train_data['id']==record_id]['discourse_text'].iloc[0].strip().replace("\xa0"," ").replace('"',''))

In [ ]:
print('Prediction string in train data should start at:', len(re.sub('\n+',' ', data).strip().replace("\xa0"," ").replace('"','')[0:re.sub('\n+',' ', data).strip().replace("\xa0"," ").replace('"','').find(' '.join(re.sub('\n+',' ',train_data[train_data['id']==record_id]['discourse_text'].iloc[0].strip().replace("\xa0"," ").replace('"','')).split(' ')[0:4]))].strip().split(' ')))

In [ ]:
len(re.sub('\n+',' ',train_data[train_data['id']==record_id]['discourse_text'].iloc[0]).strip().replace("\xa0"," ").replace('"','').split(' ')), re.sub('\n+',' ',train_data[train_data['id']==record_id]['discourse_text'].iloc[0]).strip().replace("\xa0"," ").replace('"','').split(' ')

In [ ]:
len(train_data[train_data['id']==record_id]['predictionstring'].iloc[0].split(' ')), train_data[train_data['id']==record_id]['predictionstring'].iloc[0].split(' ')

In [ ]:
train_data['cleaned_discourse_text'] = train_data['discourse_text'].apply(lambda x: re.sub('\n+',' ',x).strip().replace("\xa0"," "))
train_data['Flag - pred len and text len'] = np.where(len(train_data['cleaned_discourse_text'].str.split(' '))==len(train_data['predictionstring'].str.split(' ')), 1, 0)

mismatches_pred_string = []
records_with_name_mask = []
record_id_list = []

for i in tqdm(range(0, train_data.shape[0])):
    record_id = train_data['id'].iloc[i]
    if record_id not in record_id_list:
        with open(filepath + 'train/' + record_id+'.txt','r') as f:
            data = f.read()

        data = re.sub('\n+',' ', data).strip().replace("\xa0"," ")
        str_index = data.find(train_data[train_data['id']==record_id]['cleaned_discourse_text'].iloc[0])
        words_before_pred_string_start = data[:str_index].strip().split(' ')

        if str_index != -1:
            if (len(words_before_pred_string_start) != int(train_data['predictionstring'].iloc[i].strip().split(' ')[0])) and (words_before_pred_string_start[0] != ''):
                mismatches_pred_string.append(record_id)
            if ('PROPER_NAME' in data) or ('GENERIC_NAME' in data) or ('SCHOOL_NAME' in data):
                records_with_name_mask.append(record_id)
        else:   
            # At least first 4 words are correct
            if (len(data[:data.find(' '.join(train_data[train_data['id']==record_id]['cleaned_discourse_text'].iloc[0].split(' ')[0:4]))].strip().split(' ')) != int(train_data['predictionstring'].iloc[i].strip().split(' ')[0])) and (data[:data.find(' '.join(train_data[train_data['id']==record_id]['cleaned_discourse_text'].iloc[0].split(' ')[0:4]))].strip().split(' ')[0] != ''):
                mismatches_pred_string.append(record_id)
                if ('PROPER_NAME' in data) or ('GENERIC_NAME' in data) or ('SCHOOL_NAME' in data):
                    records_with_name_mask.append(record_id)
        record_id_list.append(record_id)

In [ ]:
print('# Records with number of words in discourse text not equal to number of indices in prediction string:', train_data[train_data['Flag - pred len and text len'] == 0].shape[0])

In [ ]:
print('# Misatches:', len(mismatches_pred_string), 'out of', train_data['id'].nunique(), 'ids in train data', '(' + str(len(mismatches_pred_string)/train_data['id'].nunique() * 100) + '%)')

In [ ]:
mismatches_pred_string

In [ ]:
len([x for x in mismatches_pred_string if x not in records_with_name_mask]), [x for x in mismatches_pred_string if x not in records_with_name_mask]

In [ ]:
record_id = '4E1C636ABEAD'

with open(filepath + 'train/' + record_id+'.txt','r') as f:
    data = f.read()

In [ ]:
train_data[train_data['id']==record_id]

In [ ]:
re.sub('\n+',' ', data).strip().replace("\xa0"," ").replace('"','')

In [ ]:
re.sub('\n+',' ',train_data[train_data['id']==record_id]['discourse_text'].iloc[4].strip().replace("\xa0"," ").replace('"',''))

In [ ]:
re.sub('\n+',' ',train_data[train_data['id']==record_id]['discourse_text'].iloc[5].strip().replace("\xa0"," ").replace('"',''))

The sentence "Therefore, driving can cause many accidents that can be fatal to the driver and passengers if there is any and cell phones should only be used when not operating a vehicle." does not have any tagged class

## Text length distribution

In [ ]:
len_text_df = pd.DataFrame(data = None, columns = ['record_id', '# Words in text'])
for i in tqdm(range(0, len(train_data['id'].unique().tolist()))):
    record_id = train_data['id'].unique().tolist()[i]
    with open(filepath + 'train/' + record_id+'.txt','r') as f:
        data = f.read()
    len_text_df.loc[i] = record_id, len(data.strip().replace("\xa0"," ").split(' '))

In [ ]:
fig = px.histogram(len_text_df, x='# Words in text', title = 'Distribution of discourse text length')
py.offline.iplot(fig)

# Modelling - NER

In [ ]:
tag2idx = {
    "B-Lead": 0,
    "I-Lead": 1,
    "B-Position": 2,
    "I-Position": 3,
    "B-Evidence": 4,
    "I-Evidence": 5,
    "B-Claim": 6,
    "I-Claim": 7,
    "B-Concluding Statement": 8,
    "I-Concluding Statement": 9,
    "B-Counterclaim": 10,
    "I-Counterclaim": 11,
    "B-Rebuttal": 12,
    "I-Rebuttal": 13,
    "O": 14
}


idx2tag = {v: k for k, v in tag2idx.items()}

In [ ]:
train_data_for_modelling = train_data[['id']].drop_duplicates().reset_index(drop = True)
train_data_for_modelling['Text'] = np.NaN
for i in tqdm(range(0, train_data_for_modelling.shape[0])):
    record_id = train_data_for_modelling['id'].iloc[i]
    with open(filepath + 'train/' + record_id+'.txt','r') as f:
        data = f.read()
    train_data_for_modelling['Text'].iloc[i] = data
    
train_data_for_modelling['cleaned_text'] = train_data_for_modelling['Text'].apply(lambda x: re.sub('\n+',' ',x).strip().replace("\xa0"," "))

In [ ]:
train_data_for_modelling.head()

In [ ]:
def tokenize_sentence(x):
    "takes in a string and returns tokenized list"

    return [x for x in x.strip().lower().split(" ") if len(x) > 0]

In [ ]:
def get_tags(sent, rec_id):
    '''
    Input: sent as a sentence tokenized as list of tokens
    output: tags 
    '''
    sent = [x for x in sent]
    data = ' '.join(sent)
    
    temp = train_data[train_data['id']==rec_id].reset_index(drop = True)
    all_tags = []
    correct_tags = []
    
    if temp.shape[0] > 0:
        for j in range(0, len(temp)):     
            pred_string = temp['discourse_text'].iloc[j]
            pred_string = re.sub('\n+', ' ', pred_string).strip().replace("\xa0"," ").lower()
            discourse_type = temp['discourse_type'].iloc[j]
            ep = tokenize_sentence(pred_string)

            start_index = data.find(pred_string)
            stop_index = data.find(pred_string) + len(pred_string)
            tokens_for_tagging = data[start_index:stop_index].split(' ')

            i = 0
            tag = []
            tag_flag = False
            while i < len(sent):
                if (start_index == 0) & (i == 0) :
                    tag.append('B-' + discourse_type)
                    i+=1

                    tag = tag + ['I-' + discourse_type] * (len(ep)-1)
                    i+=(len(ep)-1)
                    tag_flag = True
                elif (data.split(' ')[i] == tokens_for_tagging[0]) & (len(' '.join(data.split(' ')[0:i]))+1 == start_index) & (not tag_flag):
                    tag.append('B-' + discourse_type)
                    i+=1

                    tag = tag + ['I-' + discourse_type] * (len(ep)-1)
                    i+=(len(ep)-1)
                else:
                    tag.append('O')                
                    i+=1
            all_tags.append(tag)
            # Update text to mask already tagged tokens
            data = ' '.join(['masked_token' if (('B' in tag[x]) or ('I' in tag[x])) else data.split(' ')[x] for x in range(0, len(data.split(' ')))])

        for i in range(0, len(all_tags[0])):
            sum_O_tags = 0
            tags = []
            for j in range(0, len(all_tags)):
                if all_tags[j][i] == 'O':
                    sum_O_tags += 1
                else:
                    tags.append(all_tags[j][i])
            if sum_O_tags == len(all_tags):
                correct_tags.append('O')
            else:
                if len(tags)>1:
                    print('More than 1 tag for a token in record_id', rec_id)
                    print(tags)
                correct_tags.append(tags[0])

    else:
        correct_tags = ['O'] * len(sent)
    return correct_tags

In [ ]:
# # Check tagging cdde
# rec_id = '6C8D8C5145DB'
# temp = train_data[train_data['id']==rec_id].reset_index(drop = True)
# sent = train_data_for_modelling[train_data_for_modelling['id']==rec_id]['cleaned_text'].iloc[0].lower()
# data = ' '.join(sent.split(' '))
# sent = sent.split(' ')

In [ ]:
def vocab_sent_tokenize_label(sent_tokenized, token_tag):
    try:
        vocab_sent_token = []
        sent_input_ids = []
        vocab_token_tag = []
        token_tag_ids = []
        for sent_token_, tag_ in zip(sent_tokenized, token_tag):
            _vocab_sent_token = tokenizer.tokenize(sent_token_)
            _sent_input_ids = [
                tokenizer.convert_tokens_to_ids(x) for x in _vocab_sent_token
            ]
            _vocab_token_tag = [tag_] * len(_vocab_sent_token)
            _token_tag_ids = [tag2idx[x] for x in _vocab_token_tag]

            vocab_sent_token.extend(_vocab_sent_token)
            sent_input_ids.extend(_sent_input_ids)
            vocab_token_tag.extend(_vocab_token_tag)
            token_tag_ids.extend(_token_tag_ids)
        return vocab_sent_token, sent_input_ids, vocab_token_tag, token_tag_ids
    except Exception as e:
        print(f"Error in line no: {sys.exc_info()[2].tb_lineno}")
        print(e)

In [ ]:
def sent_tag_tokenization(data):
    try:
        model_data_preprocessing = data.copy(deep=True)
        model_data_preprocessing['sent_tokenized'] = model_data_preprocessing[
            'cleaned_text'].progress_apply(tokenize_sentence)

        model_data_preprocessing[
            'token_tag'] = model_data_preprocessing.progress_apply(
                lambda x: get_tags(sent=x['sent_tokenized'], rec_id=x['id'])
                if (isinstance(x['sent_tokenized'], list)) else np.nan, axis=1)

        model_data_preprocessing['vocab_sent_tokenized'], model_data_preprocessing[
            'sent_input_ids'], model_data_preprocessing[
                'vocab_token_tag'], model_data_preprocessing['token_tag_ids'] = zip(
                    *model_data_preprocessing.
                    progress_apply(lambda x: vocab_sent_tokenize_label(
                        sent_tokenized=x['sent_tokenized'], token_tag=x['token_tag'])
                                   if isinstance(x['token_tag'], list) else np.nan,
                                   axis=1))
        return model_data_preprocessing
    except Exception as e:
        print(f"Error in line no: {sys.exc_info()[2].tb_lineno}")
        print(e)

In [ ]:
def pad_data(input_ids, token_ids, max_token_length):
    try:
        attention_mask = []
        for input_ in tqdm(input_ids):
            attention_mask.append(torch.ones(len(input_[:max_token_length])))

        padded_attention_mask = torch.nn.utils.rnn.pad_sequence(attention_mask,
                                                                batch_first=True,
                                                                padding_value=0.0)

        padded_input_ids = torch.nn.utils.rnn.pad_sequence(
            [torch.tensor(input_[:max_token_length]) for input_ in input_ids],
            batch_first=True,
            padding_value=0.0)

        padded_tags = torch.nn.utils.rnn.pad_sequence(
            [torch.tensor(tag_[:max_token_length]) for tag_ in token_ids],
            batch_first=True,
            padding_value=0.0)
        return padded_input_ids, padded_attention_mask, padded_tags
    except Exception as e:
        print(f"Error in line no: {sys.exc_info()[2].tb_lineno}")
        print(e)

In [ ]:
def create_dataloader(token_ids, masks, tags, batch_size=16, val=False):
    try:
        # wrap tensors
        data = TensorDataset(token_ids, masks, tags)

        if val:
            # sampler for sampling the data during training
            sampler = SequentialSampler(data)
            
            # dataLoader for validation set
            dataloader = DataLoader(data,
                                    sampler=sampler,
                                    batch_size=batch_size)
        else:    
            # sampler for sampling the data during training
            sampler = RandomSampler(data)
            # dataLoader for train set
            dataloader = DataLoader(data,
                                    sampler=sampler,
                                    batch_size=batch_size)
        return dataloader
    except Exception as e:
        print(f"Error in line no: {sys.exc_info()[2].tb_lineno}")
        print(e)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096")

In [ ]:
# processed_tagged_full_data = sent_tag_tokenization(data=train_data_for_modelling)

In [ ]:
with open('../input/student-written-text-bio-tagged-data/processed_tagged_full_data.pkl', 'rb') as file:
    processed_tagged_full_data = pickle.load(file)

In [ ]:
# Train-validation split
train_ids, valid_ids = sklearn.model_selection.train_test_split(processed_tagged_full_data['id'].unique().tolist(), test_size = 0.2, random_state = SEED)

In [ ]:
processed_tagged_full_data.head()

In [ ]:
# with open('processed_tagged_full_data.pkl', 'wb') as file:
#     pickle.dump(processed_tagged_full_data, file)

In [ ]:
batch_size = 4
    
input_ids = processed_tagged_full_data[processed_tagged_full_data['id'].isin(train_ids)]['sent_input_ids'].tolist()
token_ids = processed_tagged_full_data[processed_tagged_full_data['id'].isin(train_ids)]['token_tag_ids'].tolist()

padded_input_ids, padded_attention_mask, padded_tags = pad_data(
    input_ids=input_ids, 
    token_ids=token_ids,
    max_token_length = 1000
)

train_dataloader = create_dataloader(
    token_ids=padded_input_ids, 
    masks=padded_attention_mask, 
    tags=padded_tags, 
    batch_size=batch_size, 
    val=False
)

input_ids = processed_tagged_full_data[processed_tagged_full_data['id'].isin(valid_ids)]['sent_input_ids'].tolist()
token_ids = processed_tagged_full_data[processed_tagged_full_data['id'].isin(valid_ids)]['token_tag_ids'].tolist()

padded_input_ids, padded_attention_mask, padded_tags = pad_data(
    input_ids=input_ids, 
    token_ids=token_ids,
    max_token_length = 1000
)

val_dataloader = create_dataloader(
    token_ids=padded_input_ids, 
    masks=padded_attention_mask, 
    tags=padded_tags, 
    batch_size=batch_size, 
    val=True
)

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
config = AutoConfig.from_pretrained(
    "allenai/longformer-base-4096", 
    num_labels=len(tag2idx),
    id2label=idx2tag,
    label2id=tag2idx
)
model = AutoModelForTokenClassification.from_pretrained("allenai/longformer-base-4096", config=config)

In [ ]:
config

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

model = model.to(device)

optimizer = AdamW(model.parameters(),
                  lr = 1e-5) # learning rate

criterion = nn.CrossEntropyLoss()

In [ ]:
def do_train(model, optimizer, loss_criteria, train_dataloader):
    try:
        model.train()

        total_loss = 0
        total_logits = []

        # iterate over batches
        for step, batch in enumerate(train_dataloader):

            # progress update after every 50 batches.
            if step % 50 == 0 and not step == 0:
                print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

            # push the batch to gpu
            batch = [r.to(device) for r in batch]

            sent_id, mask, labels = batch
            
            gc.collect()
            torch.cuda.empty_cache()
            # clear previously calculated gradients
            model.zero_grad()

            # get model predictions for the current batch
            logits = model(sent_id.to(device), mask.to(device))

            # compute the loss between actual and predicted values
            loss = loss_criteria(logits.logits.permute(0, 2, 1), labels)

            # add on to the total loss
            total_loss = total_loss + loss.item()

            # backward pass to calculate the gradients
            loss.backward()

            # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # update parameters
            optimizer.step()

            # model predictions are stored on GPU. So, push it to CPU
            logits = logits.logits.detach().cpu().numpy()

            # append the model predictions
            total_logits.append(logits)

        # compute the training loss of the epoch
        avg_loss = total_loss / len(train_dataloader)

        total_logits = np.concatenate(total_logits, axis=0)


        return avg_loss, total_logits
    except Exception as e:
        print(f"Error during training the model on line: {sys.exc_info()[2].tb_lineno}")
        print(e)


# function for evaluating the model
def do_evaluate(model, val_dataloader, loss_criteria):
    print("\nEvaluating...")
    
    # deactivate dropout layers
    model.eval()

    total_loss, total_accuracy = 0, 0

    # empty list to save the model predictions
    total_logits = []

    # iterate over batches
    for step, batch in enumerate(val_dataloader):

        # Progress update every 50 batches.
        if step % 50 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            #             elapsed = format_time(time.time() - t0)

            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))

        # push the batch to gpu
        batch = [t.to(device) for t in batch]

        sent_id, mask, labels = batch

        # deactivate autograd
        with torch.no_grad():

            # model predictions
            logits = model(sent_id.to(device), mask.to(device))

            # compute the validation loss between actual and predicted values
            loss = loss_criteria(logits.logits.permute(0, 2, 1), labels)

            total_loss = total_loss + loss.item()

            logits = logits.logits.detach().cpu().numpy()

            total_logits.append(logits)

    # compute the validation loss of the epoch
    avg_loss = total_loss / len(val_dataloader)

    # reshape the predictions in form of (number of samples, no. of classes)
    total_logits = np.concatenate(total_logits, axis=0)

    return avg_loss, total_logits

In [ ]:
%%time

# set initial loss to infinity
best_valid_loss = float('inf')

epochs=4 # Can increase this and see better performance

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]

#for each epoch

for epoch in range(epochs):
    start = time.time()
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))

    #train model
    train_loss, _ = do_train(
        model = model,
        optimizer = optimizer, 
        loss_criteria = criterion, 
        train_dataloader = train_dataloader 
    )
    
    #evaluate model
    valid_loss, _ = do_evaluate(
        model = model, 
        val_dataloader = val_dataloader, 
        loss_criteria = criterion
    )
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        model.save_pretrained('longformer_model_files')
        tokenizer.save_pretrained('tokenizer_file')
        model.config.to_json_file('config_np_new.json')

    # append training and validation loss
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)

    print(f"Time taken: {time.time() - start}")
    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')

In [ ]:
# # Load fine-tuned model
# model = AutoModelForTokenClassification.from_pretrained('../input/model_longformer')
# model = model.to(device)

In [ ]:
def get_prediction_from_logits(logits):
    try:
        tag_prob = nn.Softmax(dim=2)(logits)
        tag_prediction = torch.argmax(tag_prob, dim=2).detach().cpu().numpy()
        return tag_prediction
    except Exception as e:
        print(f"Error in line: {sys.exc_info()[2].tb_lineno}")
        print(e)
        
def classification_result(tag2idx, c_tag_id):
    try:
        prediction_result = []
        for sent_ in c_tag_id:
            prediction_result.append(
                list(map(lambda x: list(tag2idx.keys())[list(tag2idx.values()).index(x)], sent_))
            )
            
        tagged_entity = np.concatenate(prediction_result, axis=0)
        return tagged_entity
    except Exception as e:
        print(f"Error in line: {sys.exc_info()[2].tb_lineno}")
        print(e) 

In [ ]:
record_id = '423A1CA112E2' # Specify any record id for which predictions are to be seen

processed_tagged_data = sent_tag_tokenization(data=train_data_for_modelling[train_data_for_modelling['id']==record_id].reset_index(drop = True))

input_ids = processed_tagged_data['sent_input_ids'].tolist()
token_ids = processed_tagged_data['token_tag_ids'].tolist()

padded_input_ids, padded_attention_mask, padded_tags = pad_data(
    input_ids=input_ids, 
    token_ids=token_ids,
    max_token_length = 1000
)

val_dataloader = create_dataloader(
    token_ids=padded_input_ids, 
    masks=padded_attention_mask, 
    tags=padded_tags, 
    batch_size=batch_size, 
    val=True
)

for batch in val_dataloader:
    break
    
# deactivate dropout layers
model.eval()

sent_id, mask, labels = batch

# deactivate autograd
with torch.no_grad():
    # model predictions
    logits = model(sent_id.to(device), mask.to(device))
    logits = get_prediction_from_logits(logits['logits'])
    
display(train_data[train_data['id']==record_id])

print('Complete text:\n')
display(train_data_for_modelling[train_data_for_modelling['id']==record_id]['cleaned_text'].iloc[0])
print('Predicted tags:\n')
display(np.array([idx2tag[x] for x in logits[0]]))

# Predictions on Test data

In [ ]:
test_data_for_modelling = submission_data[['id']].drop_duplicates().reset_index(drop = True)
test_data_for_modelling['Text'] = np.NaN
for i in tqdm(range(0, test_data_for_modelling.shape[0])):
    record_id = test_data_for_modelling['id'].iloc[i]
    with open(filepath + 'test/' + record_id+'.txt','r') as f:
        data = f.read()
    test_data_for_modelling['Text'].iloc[i] = data
    
test_data_for_modelling['cleaned_text'] = test_data_for_modelling['Text'].apply(lambda x: re.sub('\n+',' ',x).strip().replace("\xa0"," "))

In [ ]:
test_data_for_modelling.head()

In [ ]:
processed_tagged_data = sent_tag_tokenization(data=test_data_for_modelling)

input_ids = processed_tagged_data['sent_input_ids'].tolist()
token_ids = processed_tagged_data['token_tag_ids'].tolist()

padded_input_ids, padded_attention_mask, padded_tags = pad_data(
    input_ids=input_ids, 
    token_ids=token_ids,
    max_token_length = 1000
)

batch_size = 4

test_dataloader = create_dataloader(
    token_ids=padded_input_ids, 
    masks=padded_attention_mask, 
    tags=padded_tags, 
    batch_size=batch_size, 
    val=True
)

pred_tags = []
for batch in test_dataloader:
    # deactivate dropout layers
    model.eval()

    sent_id, mask, labels = batch

    # deactivate autograd
    with torch.no_grad():
        # model predictions
        logits = model(sent_id.to(device), mask.to(device))
        logits = get_prediction_from_logits(logits['logits'])
    pred_tags.append(logits)

In [ ]:
test_data_for_modelling['Predicted tags'] = 'to_be_filled_with_predicted_tags'
pred_tags = np.concatenate(pred_tags)
for i in range(0, len(test_data_for_modelling)):
    test_data_for_modelling.at[i, 'Predicted tags'] = np.array(pred_tags[i], dtype=object)
test_data_for_modelling['Predicted tags'] = test_data_for_modelling['Predicted tags'].apply(lambda x: [idx2tag[t] for t in x])

In [ ]:
test_data_for_modelling.head()

In [ ]:
# Get prediction string as per submission format
test_data_for_submission_final = pd.DataFrame(data = None, columns = ['id','class','predictionstring','token','discourse type count']) 
for i in tqdm(range(0, len(test_data_for_modelling))):
    test_data_for_submission = pd.DataFrame(data = None, columns = ['id','class','predictionstring','token']) 
    record_id = test_data_for_modelling['id'].iloc[i]
    pred_tags = test_data_for_modelling['Predicted tags'].iloc[i]
    count = 0
    for j in range(0, len(pred_tags)):
        token = pred_tags[j]
        if token != 'O':
            discourse_type = token.split('-')[1]
            test_data_for_submission.loc[count] = [record_id, discourse_type, count, token]
            count += 1
        else:
            discourse_type = np.NaN

    count = 0
    test_data_for_submission['discourse type count'] = np.NaN
    for k in range(0, len(test_data_for_submission)):
        if test_data_for_submission['token'].iloc[k].split('-')[0] == 'B':
            discourse_type = test_data_for_submission['token'].iloc[k].split('-')[1]
            test_data_for_submission['discourse type count'].iloc[k] = count
            for j in range(k+1, len(test_data_for_submission)):
                if (test_data_for_submission['token'].iloc[j].split('-')[0] == 'I') & (test_data_for_submission['token'].iloc[j].split('-')[1] == discourse_type):
                    test_data_for_submission['discourse type count'].iloc[j] = count
                else:
                    break
            count = count + 1
    test_data_for_submission_final = pd.concat([test_data_for_submission_final, test_data_for_submission], axis = 0).reset_index(drop = True)
    
keys = test_data_for_submission_final['id'].unique().tolist()
values = [x for x in range(0, len(test_data_for_submission_final['id'].unique().tolist()))]
temp = dict(zip(keys, values))
test_data_for_submission_final['rec_num'] = test_data_for_submission_final['id'].map(temp) 

test_data_for_submission_final['predictionstring'] = test_data_for_submission_final['predictionstring'].astype(str)
test_data_for_submission_final = test_data_for_submission_final.groupby(['id','class','discourse type count','rec_num']).agg({'predictionstring': ' '.join}).reset_index().sort_values(by = ['rec_num','discourse type count'])[['id','class','predictionstring']]

In [ ]:
test_data_for_submission_final.head(10)